## Milestone 2
Dataset: Open Food Facts

The dataset is downloaded and stored in the /data folder

When describing the data, in particular, you should show (non-exhaustive list):

- That you can handle the data in its size.
- That you understand what’s into the data (formats, distributions, missing values, correlations, etc.).
- That you considered ways to enrich, filter, transform the data according to your needs.
- That you have updated your plan in a reasonable way, reflecting your improved knowledge after data acquaintance. In particular, discuss how your data suits your project needs and discuss the methods you’re going to use, giving their essential mathematical details in the notebook.
- That your plan for analysis and communication is now reasonable and sound, potentially discussing alternatives to your choices that you considered but dropped.


In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql import functions as F
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
data_folder = './data/'

# Loading the data

## Open Food Facts dataset

The data is the CSV file that can be downloaded on the openfoodfacts website. Its size is 1.6 GB. We decided for this milestone to download and load it using spark.

In [3]:
data = spark.read.option("delimiter", "\t").option("header", "true").csv(data_folder + "en.openfoodfacts.org.products.csv")

In [4]:
print('In the dataset, there are {} elements'.format(data.count()))

In the dataset, there are 697574 elements


We look at the columns and type to make sure it has been loaded correctly:

In [5]:
data.dtypes

[('code', 'string'),
 ('url', 'string'),
 ('creator', 'string'),
 ('created_t', 'string'),
 ('created_datetime', 'string'),
 ('last_modified_t', 'string'),
 ('last_modified_datetime', 'string'),
 ('product_name', 'string'),
 ('generic_name', 'string'),
 ('quantity', 'string'),
 ('packaging', 'string'),
 ('packaging_tags', 'string'),
 ('brands', 'string'),
 ('brands_tags', 'string'),
 ('categories', 'string'),
 ('categories_tags', 'string'),
 ('categories_en', 'string'),
 ('origins', 'string'),
 ('origins_tags', 'string'),
 ('manufacturing_places', 'string'),
 ('manufacturing_places_tags', 'string'),
 ('labels', 'string'),
 ('labels_tags', 'string'),
 ('labels_en', 'string'),
 ('emb_codes', 'string'),
 ('emb_codes_tags', 'string'),
 ('first_packaging_code_geo', 'string'),
 ('cities', 'string'),
 ('cities_tags', 'string'),
 ('purchase_places', 'string'),
 ('stores', 'string'),
 ('countries', 'string'),
 ('countries_tags', 'string'),
 ('countries_en', 'string'),
 ('ingredients_text', 'str

There is in fact a lot of columns that we wouldn't need, but first, let's explicity list what we need depending on the questions we had.

- Which countries are the highest exporters and importers and is there a relationship with the GDP?

For this question, what is needed in the dataset is, apart from the description/name of the food, are:
1. `origins` : origins of ingredients
2. `manufacturing_places` : places where manufactured or transformed
3. `countries` : list of countries where the product is sold

There are a lot of null values in the dataset, it is therefore a good idea to look at the number of products that have an origin or a manufacturing place:

In [9]:
print('There are {} products that contains the origins tags'.format(data.filter(data.origins != "").count()))

There are 42483 products that contains the origins tags


In [8]:
print('There are {} products that contains the manufacturing_places tags'.format(data.filter(data.manufacturing_places != "").count()))

There are 67356 products that contains the manufacturing_places tags


In [10]:
print('There are {} products that contains the countries tags'.format(data.filter(data.countries != "").count()))

There are 696976 product that contains the countries tags


As we can see, the number of products that contains the informations we need is in fact very low compared to the whole dataset. However, we still think that it is a good idea to use that subset since it can gives us a first impression of what we want. The data will need cleaning afterwards.

- How does the transport and production of food affect the ecosystem?

hmmmmmmmmmmmmmmmmmmmmmmmmmmmmm

- How does the food we eat affect our Life expectancy?

We need the following tags from our dataset:
1. `categories`

we also need the Life expectancy, that we're going to get later.

## GDP and Life Expectancy

We found the GDP per country (in USD) on the World Bank website

In [26]:
gdp = pd.read_csv(data_folder + 'GDP.csv')

In [23]:
gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.498933e+09,2.467704e+09,2.584464e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.248694e+10,1.593680e+10,1.793024e+10,2.053654e+10,2.026425e+10,2.061610e+10,1.921556e+10,1.946902e+10,2.081530e+10,NaN
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7.549239e+10,8.252614e+10,1.041158e+11,1.139232e+11,1.249125e+11,1.267302e+11,1.026212e+11,9.533720e+10,1.242094e+11,NaN
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322824e+10,1.138693e+10,1.188368e+10,1.303935e+10,NaN
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.012914e+09,NaN


Same goes for the life expectancy:

In [27]:
le = pd.read_csv(data_folder + 'LE.csv')

In [28]:
le.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,65.662,66.074,66.444,66.787,67.113,67.435,...,74.872,75.016,75.158,75.299,75.440,75.582,75.725,75.867,NaN,NaN
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.292,32.742,33.185,33.624,34.060,34.495,...,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673,NaN,NaN
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,33.251,33.573,33.914,34.272,34.645,35.031,...,57.231,58.192,59.042,59.770,60.373,60.858,61.241,61.547,NaN,NaN
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,62.279,63.298,64.187,64.911,65.461,65.848,...,76.281,76.652,77.031,77.389,77.702,77.963,78.174,78.345,NaN,NaN
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
